In [1]:
import talib as ta 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import date, timedelta
import numpy as np
import plotly.graph_objects as go 
import plotly as py
py.offline.init_notebook_mode(connected=True)


filename = '../DailyUpdate/newone/master_2021-05-25.csv'
data = pd.read_csv(filename, sep='|', header=0)
data['Date'] = pd.to_datetime(data.Date)
data = data.sort_values(by=['Date'])


#rejecting MF data
data = data[data.ClosePrice> 14]

recent_dt = max(data.Date)-timedelta(days=2)

holdings = ['AIL' ,'MHNL' ,'NICLBSL' ,'NRIC' ,'RHPL' ,'SDLBSL' ,'SJCL' ,'SMFBS' ,'TJVCL' ,'TRH' ,'UPCL' ,'UPPER','GBIME','AKPL']
##remember to sort values by company,date for callculating EMA and other indexes as the data needs to be sorted by date

In [2]:
#returns a chart wiith CP , EMA , RSI

def plot1(data,company):
#     company = 'ADBL'
    df1 = data[data.Company == company]
    df1 = df1.sort_values(by='Date')

    df1['EMA12'] = df1.groupby('Company')['ClosePrice'].apply(lambda x: ta.EMA(x,timeperiod = 12))
    df1['EMA26'] = df1.groupby('Company')['ClosePrice'].apply(lambda x: ta.EMA(x,timeperiod = 26))
    df1['RSI14'] = df1.groupby('Company')['ClosePrice'].apply(lambda x: ta.RSI(x,timeperiod = 14))
    df1['EMA20'] = df1.groupby('Company')['ClosePrice'].apply(lambda x: ta.EMA(x,timeperiod = 20))
    df1.fillna(0,inplace=True)
    
    tr1 = go.Scatter(x=df1['Date'], y=df1['ClosePrice'],mode='lines',name='CP')
    tr2 = go.Scatter(x=df1['Date'], y=df1['EMA12'],mode='lines',name='SMAlowe')
    tr3 = go.Scatter(x=df1['Date'], y=df1['EMA26'],mode='lines',name='SMAupper')
    tr4 = go.Scatter(x=df1['Date'], y=df1['RSI14'],mode='lines',name='RSI14')

    tr1['yaxis'] = 'y2'
    tr2['yaxis'] = 'y2'
    tr3['yaxis'] = 'y2'


    # fig = go.Figure(data)


    datas = [tr1,tr2,tr3,tr4]

    
    layout = {
      "yaxis": {"domain": [0, 0.23]}, 
      "yaxis2": {"domain": [0.25, 1]},
       "xaxis" : dict(
            type="category", 
            categoryorder='category ascending',
            tickmode = 'auto',nticks = 5)
    }

    fig1 = go.Figure(data = datas,layout=layout)
    fig1.update_layout(title='Share Price History {}'.format(df1['Company'].unique()[0]))
    fig1.show()
    
    return True
    
df1 = plot1(data,'NICA')


# plot1(data,'GBBL')

## recent data

In [3]:
df2 = data.sort_values(by=['Company','Date'])

df2['EMA12'] = df2.groupby('Company')['ClosePrice'].apply(lambda x: ta.EMA(x,timeperiod = 12))
df2['EMA26'] = df2.groupby('Company')['ClosePrice'].apply(lambda x: ta.EMA(x,timeperiod = 26))
df2['EMA20'] = df2.groupby('Company')['ClosePrice'].apply(lambda x: ta.EMA(x,timeperiod = 20))

df2['RSI14'] = df2.groupby('Company')['ClosePrice'].apply(lambda x: ta.RSI(x,timeperiod = 14))


#Bollingers
df2['BB_up'] = df2.groupby('Company')['ClosePrice'].apply(lambda x: ta.BBANDS(x,timeperiod = 20)[0])
df2['BB_avg'] = df2.groupby('Company')['ClosePrice'].apply(lambda x: ta.BBANDS(x,timeperiod = 20)[1])
df2['BB_low'] = df2.groupby('Company')['ClosePrice'].apply(lambda x: ta.BBANDS(x,timeperiod = 20)[2])


df2.fillna(0,inplace=True)


# (interesting_data[(interesting_data.RSI14 < 45 ) & (interesting_data.RSI14 !=0)]).sort_values(by =['Company','Date'])

# # interesting_data

# # x = (df2[(df2.Date>recent_dt) & ((df2.RSI14 >30)) &(df2.RSI14 <40) &(df2.RSI14 !=0)]).sort_values(by=['Date','ClosePrice'],ascending = False)
# # x.Company.unique()


# Level 1 : Very strong buy signals (MA crossover at 12 days) 

Level 1 : RSI < 33, Level 2 : RSI < 38, Level 3 : RSI < 45

In [4]:
# interesting_data1 = df2[(df2.Date>recent_dt) & (df2.ClosePrice > df2.EMA12)]
# (interesting_data1[(interesting_data1.RSI14 < 34 ) & (interesting_data1.RSI14 !=0)]).sort_values(by =['Company','Date'])


ma_interesting_data = df2[['Company', 'Date', 'Transactions', 'TotalSharesTraded', 'TotalAmount', 'ClosePrice', 'EMA12', 'EMA26', 'EMA20',
       'RSI14']]

# ma_interesting_data['ma_crossover_12'] = ma_interesting_data['ClosePrice'] > ma_interesting_data['EMA12']
# ma_interesting_data['prepos'] = (ma_interesting_data.groupby('Company'))['ma_crossover_12'].transform(lambda x:x.shift(1))
# ma_interesting_data.dropna(inplace=True)
# ma_interesting_data['ma_crossover_12'] = np.where(ma_interesting_data['ma_crossover_12'] == ma_interesting_data['prepos'], 0, 1)


# ma_interesting_data['ma_crossover_20'] = ma_interesting_data['EMA12'] > ma_interesting_data['EMA20']
# ma_interesting_data['prepos'] = (ma_interesting_data.groupby('Company'))['ma_crossover_20'].transform(lambda x:x.shift(1))
# ma_interesting_data.dropna(inplace=True)
# ma_interesting_data['ma_crossover_20'] = np.where(ma_interesting_data['ma_crossover_20'] == ma_interesting_data['prepos'], 0, 1)

# ma_interesting_data['ma_crossover_26'] = ma_interesting_data['EMA20'] > ma_interesting_data['EMA26']
# ma_interesting_data['prepos'] = (ma_interesting_data.groupby('Company'))['ma_crossover_26'].transform(lambda x:x.shift(1))
# ma_interesting_data.dropna(inplace=True)
# ma_interesting_data['ma_crossover_26'] = np.where(ma_interesting_data['ma_crossover_26'] == ma_interesting_data['prepos'], 0, 1)

# ma_interesting_data = ma_interesting_data[['Company', 'Date', 'Transactions', 'TotalSharesTraded', 'TotalAmount', 'ClosePrice', 'EMA12', 'EMA26', 'EMA20',
#        'RSI14','ma_crossover_12','ma_crossover_20','ma_crossover_26']]

# ma_interesting_data[(ma_interesting_data.Date>recent_dt) &( ma_interesting_data.ma_crossover_12==1)]


ma_interesting_data['ma_crossover_12'] = ma_interesting_data['ClosePrice'] > ma_interesting_data['EMA12']
ma_interesting_data['prepos'] = (ma_interesting_data.groupby('Company'))['ma_crossover_12'].transform(lambda x:x.shift(1))
ma_interesting_data.dropna(inplace=True)
ma_interesting_data['ma_crossover_12'] = np.where(ma_interesting_data['ma_crossover_12'] == ma_interesting_data['prepos'], 0, 1)

# ma_interesting_data[(ma_interesting_data.Date>recent_dt) &( ma_interesting_data.ma_crossover_12==1)]
ma_interesting_data


/Users/logpoint/Gaurav/Projects/virtualenv/lib/python3.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/logpoint/Gaurav/Projects/virtualenv/lib/python3.7/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/logpoint/Gaurav/Projects/virtualenv/lib/python3.7/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation

,Company,Date,Transactions,TotalSharesTraded,TotalAmount,ClosePrice,EMA12,EMA26,EMA20,RSI14,ma_crossover_12,prepos
445772,ACLBSL,2020-07-29,8,80.0,38160.0,492.0,0.000000,0.000000,0.000000,0.000000,0,True
445959,ACLBSL,2020-07-30,9,90.0,47690.0,541.0,0.000000,0.000000,0.000000,0.000000,0,True
221297,ACLBSL,2020-07-30,9,90.0,47690.0,541.0,0.000000,0.000000,0.000000,0.000000,0,True
446148,ACLBSL,2020-08-02,15,230.0,135750.0,595.0,0.000000,0.000000,0.000000,0.000000,0,True
221486,ACLBSL,2020-08-02,15,230.0,135750.0,595.0,0.000000,0.000000,0.000000,0.000000,0,True
...,...,...,...,...,...,...,...,...,...,...,...,...
485222,WOMI,2021-05-20,106,6425.0,11022241.0,1711.0,1710.185290,1642.358923,1678.838269,60.243101,0,True
485442,WOMI,2021-05-23,110,7117.0,12202722.0,1710.0,1710.156784,1647.369373,1681.806053,60.079564,1,True
485658,WOMI,2021-05-24,126,7657.0,13106355.0,1710.0,1710.132663,1652.008678,1684.491191,60.079564,0,False
485877,WOMI,2021-05-25,78,4317.0,7387586.0,1707.0,1709.650715,1656.082110,1686.634887,59.517429,0,False


# Level 2 : Stronger Buy Signals (MA crossover at 12 days) 


In [5]:
(ma_interesting_data[(ma_interesting_data.RSI14 < 39 ) & (ma_interesting_data.RSI14 > 33 ) & (ma_interesting_data.RSI14 !=0)]).sort_values(by =['Company','Date'])


,Company,Date,Transactions,TotalSharesTraded,TotalAmount,ClosePrice,EMA12,EMA26,EMA20,RSI14,ma_crossover_12,prepos
299640,ADBL,2010-12-01,106,4971.0,764533.0,152.0,158.419249,158.491328,159.051286,35.814823,0,False
74978,ADBL,2010-12-01,106,4971.0,764533.0,152.0,157.431672,158.010488,158.379735,35.814823,0,False
74975,ADBL,2010-12-06,11,1098.0,153720.0,145.0,148.537964,152.816815,151.828575,37.332114,0,False
299637,ADBL,2010-12-06,11,1098.0,153720.0,145.0,147.993662,152.237792,151.178234,37.332114,0,False
299629,ADBL,2010-12-16,76,3645.0,491630.0,136.0,136.964297,141.263319,139.500735,34.764700,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...
223736,WOMI,2020-08-13,70,4662.0,4156407.0,900.0,902.620174,913.731158,907.752381,38.926551,0,False
448398,WOMI,2020-08-13,70,4662.0,4156407.0,900.0,902.217071,912.714035,907.014059,38.926551,0,False
448956,WOMI,2020-08-18,24,1011.0,879968.0,873.0,881.000233,897.292246,890.279087,34.653748,0,False
224294,WOMI,2020-08-18,24,1011.0,879968.0,873.0,879.769428,895.492820,888.633460,34.653748,0,False


# Level 3 :  Buy Signals (MA crossover at 12 days) 


In [6]:
(ma_interesting_data[(ma_interesting_data.RSI14 < 45 ) & (ma_interesting_data.RSI14 > 38 ) & (ma_interesting_data.RSI14 !=0)]).sort_values(by =['Company','Date'])


,Company,Date,Transactions,TotalSharesTraded,TotalAmount,ClosePrice,EMA12,EMA26,EMA20,RSI14,ma_crossover_12,prepos
474552,ACLBSL,2021-03-03,213,6086.0,8817965.0,1464.0,1576.239405,1538.168211,1569.843306,44.887934,0,False
475806,ACLBSL,2021-03-15,125,2860.0,4134482.0,1445.0,1524.500999,1523.565511,1537.235752,44.015073,0,False
476018,ACLBSL,2021-03-16,151,5082.0,7373915.0,1444.0,1512.116230,1517.671770,1528.356156,43.944785,0,False
74980,ADBL,2010-11-29,68,2778.0,445990.0,156.0,162.388646,159.972243,161.366942,42.630387,0,False
299642,ADBL,2010-11-29,68,2778.0,445990.0,156.0,161.405777,159.678003,160.855804,42.630387,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...
223357,WOMI,2020-08-11,36,1305.0,1202913.0,910.0,905.911738,917.962905,911.353667,42.952343,0,True
223736,WOMI,2020-08-13,70,4662.0,4156407.0,900.0,902.620174,913.731158,907.752381,38.926551,0,False
448398,WOMI,2020-08-13,70,4662.0,4156407.0,900.0,902.217071,912.714035,907.014059,38.926551,0,False
451382,WOMI,2020-09-07,46,4444.0,3911957.0,876.0,886.641437,888.911591,886.907532,42.232252,0,False


# Possible Selling opportunities

In [7]:
selling_data = df2.copy()
selling_data['EMA7'] = selling_data.groupby('Company')['ClosePrice'].apply(lambda x: ta.EMA(x,timeperiod = 7))


mask = selling_data['Company'].isin(holdings)
selling_data = selling_data[mask]

selling_data2 =  selling_data[(selling_data.Date>recent_dt) & (selling_data.ClosePrice < selling_data.EMA12) ]
# selling_data2 = selling_data[(selling_data.Date>recent_dt) & (selling_data.ClosePrice < df2.EMA7)]


# Selling : RSI 

In [8]:
(selling_data2[(selling_data2.RSI14 >74 )]).sort_values(by =['Company','Date'])


,Company,Date,Transactions,TotalSharesTraded,TotalAmount,MaxPrice,MinPrice,ClosePrice,EMA12,EMA26,EMA20,RSI14,BB_up,BB_avg,BB_low,EMA7


# RSI level 2 : 60-75

In [9]:
(selling_data2[(selling_data2.RSI14 <75 ) & (selling_data2.RSI14 >60 )]).sort_values(by =['Company','Date'])


,Company,Date,Transactions,TotalSharesTraded,TotalAmount,MaxPrice,MinPrice,ClosePrice,EMA12,EMA26,EMA20,RSI14,BB_up,BB_avg,BB_low,EMA7
485562,NICLBSL,2021-05-24,1176,35799.0,63426867.0,1790.0,1760.0,1762.0,1774.149760,1681.221626,1731.534259,62.520337,1843.641100,1777.35,1711.058900,1779.032090
485782,NICLBSL,2021-05-25,939,23995.0,42364981.0,1785.0,1758.0,1758.0,1771.665181,1686.908913,1734.054806,61.831533,1843.601437,1777.50,1711.398563,1773.774067


# Bollinger data 

In [10]:
b_relevant = df2[(df2.Date>recent_dt)]
import numpy as np

b_relevant = b_relevant[['Company','Date','ClosePrice','EMA12','RSI14','BB_up','BB_avg','BB_low']]
# b_relevant['lower_closeness'] = np.log((b_relevant.ClosePrice - b_relevant.BB_low)/(b_relevant.BB_avg+b_relevant.BB_low)/2)
# b_relevant['upper_closeness'] = np.log(b_relevant.BB_up - b_relevant.BB_avg/(b_relevant.BB_up + b_relevant.BB_avg)/2)

# b_relevant['lower_closeness'] = (b_relevant.ClosePrice-b_relevant.BB_low)/(b_relevant.ClosePrice+ b_relevant.BB_low)/2
b_relevant['upper_closeness'] = (b_relevant.BB_up-b_relevant.ClosePrice)/(b_relevant.ClosePrice+b_relevant.BB_up)/2
b_relevant['mid_closeness'] = (b_relevant.BB_avg-b_relevant.ClosePrice)/(b_relevant.ClosePrice+b_relevant.BB_avg)/2

b_relevant['lower_closeness'] = (b_relevant.ClosePrice-b_relevant.BB_low)/(b_relevant.BB_avg)

b_relevant = b_relevant[(b_relevant.BB_avg !=0 ) & (b_relevant.BB_up !=0) & ((b_relevant.ClosePrice > 180))]

# 1. Close to higher band

In [11]:
b_relevant.sort_values(by='upper_closeness').head(10)

,Company,Date,ClosePrice,EMA12,RSI14,BB_up,BB_avg,BB_low,upper_closeness,mid_closeness,lower_closeness
485651,UMRH,2021-05-24,511.0,381.763830,88.172915,457.791398,330.35,202.908602,-0.027461,-0.107357,0.932621
485752,MHNL,2021-05-25,379.0,313.586847,76.486022,347.136109,294.95,242.763891,-0.021941,-0.062356,0.461896
485494,GRDBL,2021-05-24,301.0,245.891723,81.567724,278.317042,226.45,174.582958,-0.019577,-0.070670,0.558256
485669,BARUN,2021-05-25,486.0,369.466732,88.024248,454.344753,312.35,170.355247,-0.016832,-0.108756,1.010548
485535,MHNL,2021-05-24,345.0,301.693547,70.767163,326.211350,289.20,252.188650,-0.013996,-0.043992,0.320924
485870,UMRH,2021-05-25,520.0,403.030933,88.628370,492.116796,341.85,191.583204,-0.013775,-0.103353,0.960704
485446,AHPC,2021-05-24,430.0,356.028171,86.141901,407.237058,323.70,240.162942,-0.013594,-0.070519,0.586460
485832,SDLBSL,2021-05-25,1899.0,1704.022622,81.641887,1804.749805,1644.20,1483.650195,-0.012724,-0.035956,0.252615
485854,SLICL,2021-05-25,1053.0,927.027298,83.505791,1001.917844,877.50,753.082156,-0.012429,-0.045455,0.341787
485453,BARUN,2021-05-24,442.0,348.278866,85.306350,421.036298,299.80,178.563702,-0.012145,-0.095848,0.878707


# 2. Close to lower band

In [12]:
b_relevant.sort_values(by='lower_closeness').head(15)

# (b_relevant[b_relevant.ClosePrice<b_relevant.BB_avg]).sort_values(by='lower_closeness')

,Company,Date,ClosePrice,EMA12,RSI14,BB_up,BB_avg,BB_low,upper_closeness,mid_closeness,lower_closeness
485797,NUBL,2021-05-25,1739.0,1753.903308,49.330372,1784.602136,1754.75,1724.897864,0.006471,0.002254,0.008037
485783,NIFRA,2021-05-25,534.0,538.599223,45.637939,549.313489,538.65,527.986511,0.007068,0.002168,0.011164
485825,SAND2085,2021-05-25,1050.0,1049.365543,50.864135,1062.804471,1050.50,1038.195529,0.003030,0.000119,0.011237
485577,NUBL,2021-05-24,1746.0,1756.613000,50.942746,1784.355841,1755.25,1726.144159,0.005432,0.001321,0.011312
485799,PBLD84,2021-05-25,1055.0,1054.386907,50.245078,1064.424737,1053.75,1043.075263,0.002223,-0.000296,0.011316
485777,NICAD8283,2021-05-25,1122.0,1126.209869,47.169241,1145.732519,1127.15,1108.567481,0.005233,0.001145,0.011917
485606,SAND2085,2021-05-24,1051.0,1049.250187,51.797914,1062.804471,1050.50,1038.195529,0.002792,-0.000119,0.012189
485643,SRBLD83,2021-05-24,1047.0,1054.144025,45.203356,1093.795214,1063.50,1033.204786,0.010929,0.003909,0.012972
485839,SHIVM,2021-05-25,1479.0,1503.142489,47.189042,1564.167711,1511.65,1459.132289,0.013993,0.005459,0.013143
485563,NIFRA,2021-05-24,536.0,539.435445,46.790270,550.329053,539.30,528.270947,0.006595,0.001534,0.014332


# 3. Close to mid band

In [13]:
b_relevant.sort_values(by='mid_closeness',ascending=False).head(10)

,Company,Date,ClosePrice,EMA12,RSI14,BB_up,BB_avg,BB_low,upper_closeness,mid_closeness,lower_closeness
485510,JSLBB,2021-05-24,3635.0,3840.334311,33.287553,5552.706098,4204.70,2856.693902,0.104363,0.036334,0.185104
485726,JSLBB,2021-05-25,3612.0,3805.205955,32.714506,5487.830908,4145.30,2802.769092,0.103070,0.034374,0.195216
485452,API,2021-05-24,326.0,330.282956,46.251955,381.630440,341.70,301.769560,0.039308,0.011757,0.070911
485875,USLB,2021-05-25,2377.0,2446.738295,44.908750,2572.105804,2455.80,2339.494196,0.019711,0.008153,0.015272
485679,CGH,2021-05-25,1816.0,1851.560881,46.523357,1968.218015,1869.75,1771.281985,0.020112,0.007292,0.023917
485588,PRIN,2021-05-24,886.0,894.331631,45.060562,994.304262,911.00,827.695738,0.028800,0.006956,0.064000
485463,CGH,2021-05-24,1831.0,1858.026495,47.762694,1977.576604,1876.45,1775.323396,0.019243,0.006130,0.029671
485839,SHIVM,2021-05-25,1479.0,1503.142489,47.189042,1564.167711,1511.65,1459.132289,0.013993,0.005459,0.013143
485477,EIC,2021-05-24,740.0,748.207315,47.845620,799.468599,754.90,710.331401,0.019315,0.004984,0.039301
485695,FOWAD,2021-05-25,2870.0,2919.974537,48.123898,3030.233813,2920.20,2810.166187,0.013579,0.004335,0.020490


## High-low Summaries

In [14]:
#Looking at shares which are closest too the yr high value 
# 0 High_CP will mean it is closest to the HP of year

recent_dt = max(data.Date)-timedelta(days=1)

# df3 = data.sort_values(by=['Company','Date']).copy()
df2['yr_high'] = df2['ClosePrice'].rolling(window=300).max()
df2['yr_low'] = df2['ClosePrice'].rolling(window=300).min()
df2['High_CP_%'] = ((df2['yr_high'].sub(df2['ClosePrice']))/df2['yr_high'])*100

df2['Low_CP_%'] = ((df2['ClosePrice'].sub(df2['yr_low']))/df2['ClosePrice'])*100
# df2['Low_CP'] = ((df2['ClosePrice'] -df2['yr_low']))

df2.drop(columns = ['Transactions','TotalSharesTraded','TotalAmount','EMA26','MaxPrice','MinPrice'],inplace=True)

In [15]:
dx = df2[(df2.Date>recent_dt)].sort_values(by='Low_CP_%')

# dx[dx.Company !='PO']
dx[~dx['Company'].str.contains('PO')].head(10)

,Company,Date,ClosePrice,EMA12,EMA20,RSI14,BB_up,BB_avg,BB_low,yr_high,yr_low,High_CP_%,Low_CP_%
485774,NIBSF1,2021-05-25,14.05,14.894829,14.959200,28.687728,15.479918,14.9630,14.446082,468.0,14.05,96.997863,0.000000
485835,SFMF,2021-05-25,14.21,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,208.0,14.06,93.168269,1.055595
485824,SAEF,2021-05-25,15.00,14.670575,14.551810,70.307055,15.037340,14.4425,13.847660,310.0,14.01,95.161290,6.600000
485828,SBI,2021-05-25,413.00,408.072380,406.314119,56.942704,417.812191,401.4000,384.987809,542.0,380.00,23.800738,7.990315
485831,SCB,2021-05-25,603.00,594.804307,592.399324,58.978427,607.826700,585.9500,564.073300,837.0,535.00,27.956989,11.276949
485872,UNL,2021-05-25,20000.00,20280.846452,20117.158521,49.876249,21766.948911,19938.3000,18109.651089,23330.0,17453.00,14.273468,12.735000
485704,GIMES1,2021-05-25,17.12,16.619779,16.446747,73.559621,16.992634,16.4125,15.832366,2890.0,14.02,99.407612,18.107477
485706,GLH,2021-05-25,401.00,391.594065,389.039291,58.204259,409.708848,387.1000,364.491152,2200.0,328.00,81.772727,18.204489
485789,NMB,2021-05-25,431.00,426.685742,424.199292,58.261620,436.240320,420.7500,405.259680,498.0,345.00,13.453815,19.953596
485777,NICAD8283,2021-05-25,1122.00,1126.209869,1127.843918,47.169241,1145.732519,1127.1500,1108.567481,1175.0,855.00,4.510638,23.796791


In [16]:
df2[(df2.Date>recent_dt)].sort_values(by='High_CP_%',ascending=True).head(10)


,Company,Date,ClosePrice,EMA12,EMA20,RSI14,BB_up,BB_avg,BB_low,yr_high,yr_low,High_CP_%,Low_CP_%
485730,KKHC,2021-05-25,230.0,204.322006,188.233320,76.111996,251.535414,178.10,104.664586,230.0,45.20,0.0,80.347826
485816,RLFL,2021-05-25,331.0,294.828039,282.280986,72.850650,332.000000,269.20,206.400000,331.0,120.00,0.0,63.746224
485689,DHPL,2021-05-25,232.0,199.643314,184.009125,77.672812,241.930416,175.05,108.169584,232.0,47.05,0.0,79.719828
485818,RMDC,2021-05-25,1407.0,1277.539532,1245.155352,78.300022,1368.171803,1217.95,1067.728197,1407.0,550.00,0.0,60.909737
485860,SPDL,2021-05-25,423.0,374.929246,353.414014,77.246021,429.466488,342.20,254.933512,423.0,90.00,0.0,78.723404
485858,SMFDB,2021-05-25,1836.0,1710.171279,1654.189373,72.966660,1856.938496,1623.55,1390.161504,1836.0,499.00,0.0,72.821351
485701,GHL,2021-05-25,247.0,215.880243,203.930490,75.160979,248.208153,197.30,146.391847,247.0,51.00,0.0,79.352227
485771,NHPC,2021-05-25,214.0,191.617364,183.216609,74.182515,215.902869,176.25,136.597131,214.0,49.00,0.0,77.102804
485769,NGPL,2021-05-25,370.0,327.938923,311.609654,73.458513,372.218206,305.20,238.181794,370.0,100.00,0.0,72.972973
485854,SLICL,2021-05-25,1053.0,927.027298,898.695159,83.505791,1001.917844,877.50,753.082156,1053.0,360.00,0.0,65.811966


In [17]:
(data[data.Date>recent_dt]).sort_values(by='TotalSharesTraded',ascending=False).head(20)

,Company,Date,Transactions,TotalSharesTraded,TotalAmount,MaxPrice,MinPrice,ClosePrice
485771,NHPC,2021-05-25,2363,897010.0,1.875307e+08,215.00,200.00,214.00
485665,AKPL,2021-05-25,3071,825687.0,3.383376e+08,424.00,390.00,421.00
485869,UMHL,2021-05-25,1433,610392.0,1.802782e+08,305.00,282.00,300.00
485712,HDHPC,2021-05-25,1559,585162.0,1.333592e+08,235.00,218.00,233.00
485873,UPCL,2021-05-25,1681,545952.0,1.477470e+08,279.00,248.00,279.00
485715,HIDCL,2021-05-25,3644,522263.0,3.034227e+08,599.00,572.00,574.00
485668,API,2021-05-25,1904,448920.0,1.511903e+08,346.00,324.00,342.00
485704,GIMES1,2021-05-25,64,418017.0,7.143565e+06,17.20,16.90,17.12
485864,SSHL,2021-05-25,1211,411162.0,1.192171e+08,303.00,276.00,293.00
485835,SFMF,2021-05-25,49,400530.0,5.663290e+06,14.26,13.91,14.21


# -----

In [18]:
import warnings
warnings.filterwarnings('ignore')

datax = data.copy()
# datax = datax[datax.Company=='NICA']

# datax = data[data.Date>'2020']
datax = datax.sort_values(by = ['Company','Date'])
datax['P_DI'] = datax.groupby('Company')[['ClosePrice','MaxPrice','MinPrice']].apply(lambda x: (ta.PLUS_DI(x['MaxPrice'],x['MinPrice'],x['ClosePrice'],timeperiod=12).to_frame()))
datax['M_DI'] = datax.groupby('Company')[['ClosePrice','MaxPrice','MinPrice']].apply(lambda x: (ta.MINUS_DI(x['MaxPrice'],x['MinPrice'],x['ClosePrice'],timeperiod=26).to_frame()))



datax = datax.sort_values(by = ['Company','Date'])
# datax.tail(14)

In [19]:
datax['indicator'] = datax['P_DI'] > datax['M_DI']



datax['prepos'] = (datax.groupby('Company'))['indicator'].transform(lambda x:x.shift(1))
datax.dropna(inplace=True)
datax['crossover'] = np.where(datax['indicator'] == datax['prepos'], 0, 1)


# import pandas_ta as pta
# (pta.adx(datax['MaxPrice'],datax['MinPrice'],datax['ClosePrice'],length = 14)).tail(10)

In [20]:
datax[(datax.Date > recent_dt) & (datax.crossover==1) & (datax.TotalSharesTraded > 700)]

,Company,Date,Transactions,TotalSharesTraded,TotalAmount,MaxPrice,MinPrice,ClosePrice,P_DI,M_DI,indicator,prepos,crossover
485728,KBLPO,2021-05-25,2,48029.0,5571364.0,116.0,116.00,116.00,42.813333,42.777205,True,False,1
485747,MEGAPO,2021-05-25,1,11729.0,1430938.0,122.0,122.00,122.00,36.078321,37.364065,False,True,1
485774,NIBSF1,2021-05-25,18,52362.0,736167.9,14.1,14.01,14.05,15.118641,19.449390,False,True,1
485808,PRIN,2021-05-25,237,31614.0,28330138.0,902.0,875.00,901.00,21.766348,22.544009,False,True,1
485839,SHIVM,2021-05-25,400,37331.0,55423795.0,1518.0,1460.00,1479.00,19.136497,19.281188,False,True,1


# Fundamental Analysis


In [21]:
import datetime
curr = str(datetime.datetime.now()).split(' ')[0]

fundamentals = pd.read_csv('../fundamentals2.csv', sep=',', header=0)
dfx = df2[(df2.Date > recent_dt)]
dfx = dfx[['Company', 'Date', 'ClosePrice', 'RSI14',
       'yr_high', 'yr_low']]

fundamentals = fundamentals[fundamentals.Quarter == '2076 Quarter 4']



In [22]:
new = pd.merge(dfx,fundamentals,on = 'Company')
new = new[(new.Date > recent_dt) & (new.EPS >0)]
new['price_nw_timeeps'] = (new['Networth']/new['ClosePrice'])*new['EPS']
new['yearly_return'] = (100/new['ClosePrice'])*new['EPS']
new.sort_values(by = 'price_nw_timeeps',ascending =False).head(10)

p = new.copy()

In [23]:
new = new[(new.yearly_return > 0) & (new.Date > recent_dt)]
(new.sort_values(by = 'yearly_return',ascending = False)).head(10)

,Company,Date,ClosePrice,RSI14,yr_high,yr_low,Paidup,Reserve,EPS,PE,Networth,Quarter,price_nw_timeeps,yearly_return
0,ADBL,2021-05-25,446.0,65.087792,554.0,329.0,1.498917e+10,1.136981e+10,32.88,13.35,237.54,2076 Quarter 4,17.511917,7.372197
6,CBL,2021-05-25,267.0,71.339395,273.0,129.0,1.398525e+10,4.795694e+09,16.39,17.82,146.91,2076 Quarter 4,9.018183,6.138577
5,BPCL,2021-05-25,479.0,72.857589,479.0,312.0,7.218451e+09,0.000000e+00,26.94,14.88,268.96,2076 Quarter 4,15.126894,5.624217
38,NBL,2021-05-25,438.0,57.317484,477.0,230.0,1.128282e+10,1.298544e+10,22.66,14.39,267.50,2076 Quarter 4,13.839155,5.173516
32,MDB,2021-05-25,469.0,68.029272,499.0,301.0,6.933713e+08,3.108064e+08,24.14,15.12,144.83,2076 Quarter 4,7.454576,5.147122
4,BOKL,2021-05-25,341.0,66.016844,353.0,199.0,8.546887e+09,4.808473e+09,17.26,16.34,167.44,2076 Quarter 4,8.475116,5.061584
57,SBL,2021-05-25,440.0,60.627068,472.0,264.0,9.787767e+09,5.243364e+09,21.38,14.83,163.22,2076 Quarter 4,7.931008,4.859091
63,SRBL,2021-05-25,332.0,63.445041,352.0,213.0,8.967811e+09,3.469796e+09,15.53,15.78,150.86,2076 Quarter 4,7.056795,4.677711
28,LBL,2021-05-25,331.0,64.067081,339.0,189.0,9.812560e+09,2.544326e+09,15.13,16.52,142.24,2076 Quarter 4,6.501786,4.570997
55,SANIMA,2021-05-25,428.0,67.755570,437.0,305.0,8.801381e+09,2.825687e+09,19.35,17.62,144.83,2076 Quarter 4,6.547805,4.521028


In [24]:
p = new[(new.ClosePrice < 900) & (new.Date =='2020-10-06')]
(p.sort_values(by = 'EPS',ascending = False)).head(10)

,Company,Date,ClosePrice,RSI14,yr_high,yr_low,Paidup,Reserve,EPS,PE,Networth,Quarter,price_nw_timeeps,yearly_return


In [25]:
new['Grahams'] = (22.5*new.EPS * new.Networth)**0.5

new['diff'] = new['Grahams'] - new['ClosePrice']
new = new[new.Grahams > new.ClosePrice][['Company','ClosePrice','Grahams','EPS','yr_high','yr_low','Networth','diff']]
new.sort_values(by = 'diff',ascending = False)


,Company,ClosePrice,Grahams,EPS,yr_high,yr_low,Networth,diff


# Just random try section

In [26]:
import datetime
curr = str(datetime.datetime.now()).split(' ')[0]

fundamentals = pd.read_csv('../fundamentals2.csv', sep=',', header=0)
dfx = df2[(df2.Date > recent_dt)]
dfx = dfx[['Company', 'Date', 'ClosePrice', 'RSI14',
       'yr_high', 'yr_low']]

# fundamentals = fundamentals[fundamentals.Quarter == '2076 Quarter 4']

new = pd.merge(dfx,fundamentals,on = 'Company')
new = new[(new.Date > recent_dt) & (new.EPS >0)]
new['price_nw_timeeps'] = (new['Networth']/new['ClosePrice'])*new['EPS']
new['yearly_return'] = (100/new['ClosePrice'])*new['EPS']

new['exp'] = new['Paidup']/new['EPS']

p = new.sort_values(by = 'exp',ascending =True).head(20)
p = p[p.Paidup < 202100000.0]
p['Low_CP_%'] = ((p['ClosePrice'].sub(p['yr_low']))/p['ClosePrice'])*100
p.sort_values(by = 'Low_CP_%',ascending =True).head(20)


,Company,Date,ClosePrice,RSI14,yr_high,yr_low,Paidup,Reserve,EPS,PE,Networth,Quarter,price_nw_timeeps,yearly_return,exp,Low_CP_%
145,UNL,2021-05-25,20000.0,49.876249,23330.0,17453.0,92100000.0,2.232300e+09,386.00,46.37,2142.00,2076 Quarter 4,41.340600,1.930000,2.386010e+05,12.735000
54,KMCDB,2021-05-25,1501.0,59.263974,1577.0,690.0,108000000.0,1.054780e+08,27.72,26.52,197.67,2076 Quarter 1,3.650508,1.846769,3.896104e+06,54.030646
148,VLBS,2021-05-25,1873.0,56.141799,1925.0,832.0,194810000.0,1.084809e+08,46.20,19.81,155.69,2076 Quarter 1,3.840298,2.466631,4.216667e+06,55.579285
91,NMBMF,2021-05-25,1570.0,60.792523,1590.0,680.0,162006000.0,1.626410e+08,37.74,22.10,200.39,2076 Quarter 1,4.817018,2.403822,4.292687e+06,56.687898
131,SLBSL,2021-05-25,1600.0,65.203792,1693.0,686.0,100000000.0,8.313362e+07,26.41,29.80,183.13,2076 Quarter 1,3.022790,1.650625,3.786445e+06,57.125000
37,GMFBS,2021-05-25,1800.0,63.346675,1864.0,730.0,103500000.0,2.215717e+07,21.12,39.63,121.41,2076 Quarter 1,1.424544,1.173333,4.900568e+06,59.444444
72,MMFDB,2021-05-25,1747.0,55.484910,1805.0,682.0,128786400.0,1.163080e+08,66.37,13.91,190.31,2076 Quarter 1,7.230037,3.799084,1.940431e+06,60.961649
68,MFIL,2021-05-25,630.0,77.217207,646.0,229.0,800000.0,3.103070e+05,8.45,19.29,147.90,2076 Quarter 4,1.983738,1.341270,9.467456e+04,63.650794
51,JSLBB,2021-05-25,3612.0,32.714506,5395.0,1310.0,39060600.0,5.903771e+07,65.22,20.76,258.16,2075 Quarter 3,4.661460,1.805648,5.989052e+05,63.732004
110,RLFL,2021-05-25,331.0,72.850650,331.0,120.0,832416.0,3.073210e+05,14.95,11.24,128.13,2076 Quarter 2,5.787140,4.516616,5.568000e+04,63.746224


In [27]:
dfx[dfx.RSI14 >0].sort_values(by='RSI14').head(20)

,Company,Date,ClosePrice,RSI14,yr_high,yr_low
485774,NIBSF1,2021-05-25,14.05,28.687728,468.0,14.05
485726,JSLBB,2021-05-25,3612.00,32.714506,5395.0,1310.00
485747,MEGAPO,2021-05-25,122.00,42.401692,338.0,100.00
485875,USLB,2021-05-25,2377.00,44.908750,2761.0,290.00
485783,NIFRA,2021-05-25,534.00,45.637939,1899.0,292.00
485679,CGH,2021-05-25,1816.00,46.523357,2625.0,118.00
485851,SLBS,2021-05-25,1835.00,46.691370,2275.0,262.00
485777,NICAD8283,2021-05-25,1122.00,47.169241,1175.0,855.00
485839,SHIVM,2021-05-25,1479.00,47.189042,1670.0,459.00
485695,FOWAD,2021-05-25,2870.00,48.123898,3348.0,368.00
